<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/light_20240827.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import copy
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

# warningsを非表示にする
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:

# !pip install catboost

In [3]:
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',100)

In [4]:

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
#from catboost import CatBoostClassifier, Pool

In [12]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata2/df_train_allnum7.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata2/df_test_allnum7.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [13]:
df_train.head()

,id,Age,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,Passport,PitchSatisfactionScore,MonthlyIncome,ProdTaken,OwnCar,NumberOfChildren,HasChildren,TypeofContact_Company Invited,TypeofContact_Self Enquiry,TypeofContact_nan,CityTier_1,CityTier_2,CityTier_3,Occupation_Large Business,Occupation_Salaried,Occupation_Small Business,ProductPitched_BASIC,ProductPitched_DELUXE,ProductPitched_KING,ProductPitched_STANDARD,ProductPitched_SUPERDELUXE,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚
0,0,1.149839,0.065271,0,-1.908669,0.360305,3.0,1.033297,1,4,-1.334334,1.0,0,-1.300936,0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,1.767425,-0.078771,0,-1.908669,0.360305,3.0,-0.695839,1,4,0.647770,0.0,1,-1.300936,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,2,0.223460,-0.654942,1,-1.908669,-0.743578,3.0,0.456918,0,4,-1.015239,1.0,0,-1.300936,0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,-0.188264,0.497399,1,-1.908669,-0.743578,4.0,-1.272218,0,5,-0.374678,0.0,1,-1.300936,0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,4,0.943977,0.353356,1,-1.908669,-0.743578,4.0,0.456918,0,4,-1.274701,1.0,1,-1.300936,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [8]:
# Config cell
target = "ProdTaken"

In [37]:

# カラム名の変更マッピング
column_mapping = {
    'Designation_Senior Manager': 'Designation_Senior_Manager',
    'TypeofContact_Company Invited': 'TypeofContact_Company_Invited',
    'TypeofContact_Self Enquiry': 'TypeofContact_Self_Enquiry',
    'Occupation_Large Business': 'Occupation_Large_Business',
    'Occupation_Small Business': 'Occupation_Small_Business',
}
df_train.rename(columns=column_mapping, inplace=True)
df_test.rename(columns=column_mapping, inplace=True)

In [22]:
default_categorical_features = ['TypeofContact', 'CityTier', 'Occupation', 'Gender', 'ProductPitched', 'PreferredPropertyStar',
                                'Passport', 'PitchSatisfactionScore', 'Designation', 'MaritalStatus']
default_numerical_features = ['id','Age', 'DurationOfPitch', 'MonthlyIncome',
                            'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren','OwnCar'
                              ]
added_numerical_features = ['IncomePerAge','IncomePerTrips','FamilySize','PersonTripsProduct','IncomePerFamilySize','IncomePerChild','LogMonthlyIncome','PitchFollowupInteraction','TripsPerAge','FollowupPerTrip']
#numerical_features = default_numerical_features + added_numerical_features
numerical_features = default_numerical_features


#特徴量の指定
features = df_train.columns.tolist()
#カテゴリカル特徴量の指定
categorical_features = copy.deepcopy(features)
print(f'numerical_features {numerical_features}')
for i in numerical_features:
    categorical_features.remove(i)
print(f'categorical_features {categorical_features}')

#学習に使用しない特徴量は以下で除外
RemoveList=[target,'id']
for i in RemoveList:
    if i in numerical_features:
        numerical_features.remove(i)
        features.remove(i)
    elif i in categorical_features:
        categorical_features.remove(i)
        features.remove(i)
    else:
        print(f'{i} is not in numerical_features or categorical_features')
        features.remove(i)
print(f'features for training:{features}')

for col in categorical_features:
    df_train[col] = df_train[col].astype(int)
    df_test[col] = df_test[col].astype(int)

numerical_features ['id', 'Age', 'DurationOfPitch', 'MonthlyIncome', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'NumberOfTrips', 'NumberOfChildren', 'OwnCar']
categorical_features ['Gender', 'PreferredPropertyStar', 'Passport', 'PitchSatisfactionScore', 'ProdTaken', 'HasChildren', 'TypeofContact_Company Invited', 'TypeofContact_Self Enquiry', 'TypeofContact_nan', 'CityTier_1', 'CityTier_2', 'CityTier_3', 'Occupation_Large Business', 'Occupation_Salaried', 'Occupation_Small Business', 'ProductPitched_BASIC', 'ProductPitched_DELUXE', 'ProductPitched_KING', 'ProductPitched_STANDARD', 'ProductPitched_SUPERDELUXE', 'Designation_AVP', 'Designation_Executive', 'Designation_Manager', 'Designation_Senior_Manager', 'Designation_VP', 'MaritalStatus_未婚', 'MaritalStatus_独身', 'MaritalStatus_結婚', 'MaritalStatus_離婚']
features for training:['Age', 'DurationOfPitch', 'Gender', 'NumberOfPersonVisiting', 'NumberOfFollowups', 'PreferredPropertyStar', 'NumberOfTrips', 'Passport', 'PitchSatisfactionScore

In [24]:
cols_category=categorical_features

In [25]:
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [45]:


params_lgb = {
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 20,
    'feature_fraction': 0.4,
    'bagging_fraction': 0.8,
    'bagging_freq': 3,
    'verbose': -1,
    'lambda_l1': 0.80,
    'lambda_l2': 40,
    "n_estimators": 3000,
    "learning_rate": 0.01,
    "subsample_freq": 1,
    "subsample": 0.8,
    "random_seed": 0,
}

In [27]:

# f1スコアが最も高くなる点を見つける
from sklearn import metrics
def decide_cutoff(val_y, preds_y_proba):
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        #preds_y=preds_y_proba
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]

In [46]:
list_metrics_auc = []
list_metrics_f1 = []
list_cutoff = []
list_models = []
list_feature_importances = []  # 特徴量重要度を格納するリスト

cv = StratifiedKFold(n_splits=7, shuffle=True, random_state=42)
for fold, (trn_idx, val_idx) in enumerate(cv.split(train_x, train_y), start=1):
    trn_x = train_x[features].iloc[trn_idx, :]
    trn_y = train_y[trn_idx]
    val_x = train_x[features].iloc[val_idx, :]
    val_y = train_y[val_idx]

    # LightGBMデータセットの作成
    train_data = lgb.Dataset(data=trn_x, label=trn_y, categorical_feature=cols_category)
    val_data = lgb.Dataset(data=val_x, label=val_y, categorical_feature=cols_category, reference=train_data)

    model_lgb = lgb.LGBMClassifier(**params_lgb)
    model_lgb.fit(
        trn_x, trn_y,
        eval_set=(val_x, val_y),
        callbacks=[lgb.early_stopping(100, verbose=True)],
        categorical_feature=cols_category,
    )

    list_models.append(model_lgb)
    preds_y_proba = model_lgb.predict_proba(val_x)[:, 1]
    auc = roc_auc_score(val_y, preds_y_proba)
    f1, threshold = decide_cutoff(val_y, preds_y_proba)
    list_metrics_auc.append(auc)
    list_metrics_f1.append(f1)
    list_cutoff.append(threshold)
    print(f"Fold: {fold}, AUC: {auc}, F1 score: {f1}, Threshold: {threshold}")

    # 特徴量の重要度を追加
    feature_importances = model_lgb.feature_importances_
    list_feature_importances.append(feature_importances)

# AUCの出力
print("Mean AUC:", np.mean(list_metrics_auc))
print("Mean F1 Score:", np.mean(list_metrics_f1))
print("Median Cutoff:", np.median(list_cutoff))

# 特徴量重要度の集計
mean_feature_importances = np.mean(list_feature_importances, axis=0)

# 結果の表示
feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Mean Importance': mean_feature_importances
}).sort_values(by='Mean Importance', ascending=False)

print("Feature Importances:")
print(feature_importance_df)

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[529]	valid_0's auc: 0.856062
Fold: 1, AUC: 0.8560616032644465, F1 score: 0.7493820473681618, Threshold: 0.3402007525655415
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[983]	valid_0's auc: 0.829341
Fold: 2, AUC: 0.8293405291562458, F1 score: 0.7269360799394308, Threshold: 0.2506411243014924
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[661]	valid_0's auc: 0.845038
Fold: 3, AUC: 0.8450375148084771, F1 score: 0.7275829125153541, Threshold: 0.3185863626927826
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[1102]	valid_0's auc: 0.832998
Fold: 4, AUC: 0.8329979879275654, F1 score: 0.7397536394176931, Threshold: 0.26725062461586196
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[676]	valid_0's a

In [ ]:
00.8409536317909646

SyntaxError: invalid syntax (<ipython-input-81-76ea1f53a6c0>, line 1)

In [ ]:
0.8437365146029495

In [30]:
import pickle

In [47]:
# モデルを保存する
# 全てのモデルを保存する
for fold, model_cat in enumerate(list_models, start=1):
    with open(f'/content/drive/MyDrive/Signate/2024summer/models/lightgbm_fold_v11{fold}.pkl', 'wb') as f:
        pickle.dump(model_cat, f)

In [48]:
threshold = np.median(list_cutoff)
preds_y_proba = np.zeros(len(df_test))
for model in list_models:
    preds_y_proba += model.predict_proba(df_test[model.feature_name_])[:, 1] / len(list_models)
    print(f"Model: {model}, Proba: {preds_y_proba}")  # 予測確率を表示
preds_y = preds_y_proba


Model: LGBMClassifier(bagging_fraction=0.8, bagging_freq=3, feature_fraction=0.4,
               lambda_l1=0.8, lambda_l2=40, learning_rate=0.01, metric='auc',
               n_estimators=3000, num_leaves=20, objective='binary',
               random_seed=0, subsample=0.8, subsample_freq=1, verbose=-1), Proba: [0.01318831 0.02532176 0.03871153 ... 0.08116694 0.03397358 0.00379501]
Model: LGBMClassifier(bagging_fraction=0.8, bagging_freq=3, feature_fraction=0.4,
               lambda_l1=0.8, lambda_l2=40, learning_rate=0.01, metric='auc',
               n_estimators=3000, num_leaves=20, objective='binary',
               random_seed=0, subsample=0.8, subsample_freq=1, verbose=-1), Proba: [0.0270512  0.05597516 0.0816533  ... 0.17421061 0.06976499 0.00557609]
Model: LGBMClassifier(bagging_fraction=0.8, bagging_freq=3, feature_fraction=0.4,
               lambda_l1=0.8, lambda_l2=40, learning_rate=0.01, metric='auc',
               n_estimators=3000, num_leaves=20, objective='binary',
   

In [39]:
count = 0

In [49]:

from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/lightgbm_{timestamp}_{count:03d}.csv"
ss[1] = preds_y
ss.to_csv(file_name, header=False, index=False)